# 実験実行環境を再構築した場合のセットアップを行う

実験パッケージのREADME.mdから、実験実行環境の再構築した場合に必要な初期セットアップを行います。  
上から順番に実行してください。

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。  
このタスクでは、表示された名前の研究リポジトリの配下に新しく実験パッケージを作成します。  

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('blue')

## 1. ユーザー認証を行う

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行し、画面の表示に沿ってGIN-forkに登録したユーザー名、パスワード、メールアドレスを入力してください。各項目入力後、Enterキーを押下してください。  

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

tokens = []
access_token = {}

# 正常に認証が終わるまで繰り返し
os.chdir(path_flow_root)
from util.scripts import utils
tokens, access_token, name, email = utils.verify_GIN_user()

os.chdir(os.environ['HOME'])
!git config --global user.name $name
!git config --global user.email $email
clear_output()
print("認証が正常に完了しました。次の手順へお進みください。")

## 2. データ同期のための設定をする

この手順では、今の実行環境とGIN-forkのリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json

os.chdir(path_flow_root)
from util.scripts import utils

pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'/api/v1/user/keys?token=' + tokens[0]['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

# コンテナを消す際にコンテナとつなぐための公開鍵も削除のため、
# パラメータとしてGINから発行された鍵IDを保存
if response.status_code == HTTPStatus.CREATED:
    # params.jsonへの追記（鍵ID）
    params['ginKeyId'] = str(response.json()['id'])
    with open(utils.fetch_param_file_path(), mode='w') as f:
        json.dump(params, f, indent=4)
    print('公開鍵の用意が出来ました。')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('すでに公開鍵の用意が完了しています。')

In [ ]:
import os
import subprocess
from datalad import api
from IPython.display import clear_output

# sibling url をsshに変更する
os.chdir(os.environ['HOME'])
http_url = subprocess.getoutput('git config --get remote.origin.url')
ssh_url = http_url.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])
    
# siblingsにGINを登録する
sibling = !datalad siblings -s gin
for item in sibling:
    if 'unknown sibling name' in item:
        api.siblings(action='add', name='gin', url=ssh_url)
    else:
        pass

clear_output()
print('SSH接続の準備が完了しました。')

## 3. 当実行環境で編集する実験パッケージを選択する

以下のセルを実行して、この実験実行環境で編集したい実験パッケージを1つ選択してください。  
選択を間違えた場合は、再度このセルから実行して、選択しなおしてください。

In [ ]:
import json
from ipywidgets import Dropdown, Button
from IPython.display import clear_output

with open('/home/jovyan/pipeline.json', 'r') as f:
    pipeline = json.load(f)

def on_click_callback(clicked_button: Button) -> None:
    global package
    package=dropdown.value
    clear_output()
    print("入力を受けつけました：", package)
    EXPERIMENT_TITLE = experiment_title
    %store EXPERIMENT_TITLE

dropdown = Dropdown(
    options=pipeline,
    description='実験パッケージ名:',
    disabled=False,
)

button = Button(description='入力完了')
button.on_click(on_click_callback)
print("実験パッケージの選択後、入力完了ボタンを押下してください。")
display(dropdown, button)

## 4. 当実験パッケージのデータを用意する

当実験パッケージのデータをこの実験実行環境に用意します。

In [ ]:
import os

os.chdir(os.environ['HOME'] + '/experiments/' + package)
!datalad get .

## 5. 本タスクの実行結果をGIN-forkに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
os.chdir(path_flow_root)
from util.scripts import utils

# SSHホスト（＝GIN）を信頼する設定
utils.config_GIN(ginHttp = params['siblings']['ginHttp'])

# S3にあるデータをGIN-forkに同期しないための設定
!git annex untrust here
!git annex --force trust web

In [ ]:
import os
import glob
  
os.chdir(experiment_path)

#**************************************************#
#* Generate a list of folder paths to be managed by Git-annex. #
#**************************************************#
dirlist=[]
filelist=[]
annexed_save_path=[]

# Recursively search under the experimental package to obtain a list of absolute directory paths.
for root, dirs, files in os.walk(top=experiment_path):
    for dir in dirs:
        dirPath = os.path.join(root, dir)
        dirlist.append( dirPath )   

# Add directory paths containing the string "output_data" that are not included under input_data to annexed_save_path.
output_data_path = [ s for s in dirlist if 'output_data' in s ]
for output_data in output_data_path:  
    if  "input_data" not in output_data:
        annexed_save_path.append( output_data )

# Add the input_data directory to annexed_save_path.
annexed_save_path.append( experiment_path + '/input_data'  )

# Generate a list of file paths to which metadata is to be assigned.
gitannex_files = []
for path in annexed_save_path:
    gitannex_files += [p for p in glob.glob(path+'/**', recursive=True)
             if os.path.isfile(p)]

#********************************************************#
#* Generate a list of directory paths and file paths to be managed by Git. #
#********************************************************#
# Obtain a list of directories and files directly under the experimental package.
files = os.listdir()

# Delete Git-annex managed directories (input_data and output_data) from the retrieved list.
dirs = [f for f in files if os.path.isdir(f)]

for dirname in dirs:
    if dirname == 'input_data' :
        dirs.remove('input_data')

    if dirname == 'output_data' :
        dirs.remove('output_data')
    
# Obtain files directly under the experimental package.
files = [f for f in files if os.path.isfile(f)]

# Generate a list of folder paths and file paths to be managed by Git.
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)
save_path.append('/home/jovyan/pipeline.json')


以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC%E6%A9%9F%E8%83%BD#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

utils.syncs_with_repo(git_path=save_path, gitannex_path=annexed_save_path, gitannex_files=gitannex_files, message=EXPERIMENT_TITLE + '_ワークフロー実行準備')

## 6. 実験ワークフロー機能トップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして実験ワークフロー機能トップページに戻ってください。  

from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../experiment.ipynb'>実験ワークフロー機能トップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))